In [1]:
from pulp import *
import pandas as pd
import plotly.express as px
from models import *
import numpy as np

## Load in data

In [2]:
def load_res_profile(file:str)->pd.Series:

    res_norm = pd.read_csv(f'data/renewable_profiles/{file}',skiprows=3,index_col=0)
    res_norm.index = pd.to_datetime(res_norm.index)
    res_norm = res_norm['electricity']
    return res_norm

wind_profile = 'wind_amsterdam.csv'
solar_profile = 'solar_amsterdam.csv'

wind = load_res_profile(wind_profile)
solar = load_res_profile(solar_profile)

In [3]:
res = Renewables(
    names=['Solar','Wind'],
    capacities={'Solar':100,'Wind':300},
    profiles=pd.concat([wind,solar],axis=1,keys=['Wind','Solar'])
)

res.generation

Solar 100
Wind 300


,Solar,Wind,Total
time,,,
2019-01-01 00:00:00,0.0,185.7,185.7
2019-01-01 01:00:00,0.0,192.9,192.9
2019-01-01 02:00:00,0.0,196.5,196.5
2019-01-01 03:00:00,0.0,212.4,212.4
2019-01-01 04:00:00,0.0,223.8,223.8
...,...,...,...
2019-12-31 19:00:00,0.0,46.2,46.2
2019-12-31 20:00:00,0.0,45.9,45.9
2019-12-31 21:00:00,0.0,46.5,46.5


In [4]:
time_index = range(res.generation.shape[0])
time_index

range(0, 8760)

## Define load and renewables size (MW)

In [5]:
load = Load(
    capacity=80,
    profile=pd.Series(np.ones(len(time_index)),index=res.generation.index)
)
load.load

time
2019-01-01 00:00:00    80.0
2019-01-01 01:00:00    80.0
2019-01-01 02:00:00    80.0
2019-01-01 03:00:00    80.0
2019-01-01 04:00:00    80.0
                       ... 
2019-12-31 19:00:00    80.0
2019-12-31 20:00:00    80.0
2019-12-31 21:00:00    80.0
2019-12-31 22:00:00    80.0
2019-12-31 23:00:00    80.0
Length: 8760, dtype: float64

In [6]:
storage = Storage(
    efficiency=0.7,
    minimum_duration=10,
    capex_power=800,
    capex_capacity=100,
)

In [7]:
inputs = ScenarioInputs(
    name='BaseCase',
    time_index=time_index,
    renewables =res,
    load=load,
    storage=storage

)

In [11]:
scenario = Model(inputs=inputs)

In [13]:
scenario.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/andrewmoore/.pyenv/versions/3.8.12/envs/py3-twaice/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/69/tcb5hky53c9d4y57_lvg5f2r0000gn/T/431031641b2b46e58de608989319a841-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/69/tcb5hky53c9d4y57_lvg5f2r0000gn/T/431031641b2b46e58de608989319a841-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 43806 COLUMNS
Duplicate row C0000001 at line 70097 <     X0000004  C0000001   1.000000000000e+00 >
Duplicate row C0000002 at line 70098 <     X0000004  C0000002  -7.000000000000e-01 >
Duplicate row C0008761 at line 70099 <     X0000004  C0008761   1.000000000000e+00 >
Duplicate row C0008762 at line 70100 <     X0000004  C0008762   1.000000000000e+00 >
Duplicate row C0008763 at line 70101 <     X0000004  C0008763   1.000000000000e+00 >
Duplicate row C0008764 at line 70102 <

PulpSolverError: Pulp: Error while executing /Users/andrewmoore/.pyenv/versions/3.8.12/envs/py3-twaice/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc

## Results

In [ ]:
for var in scenario.model.variables():
    if var.name in ['capacity','power']:
        print(f"{var.name}: {var.value()}")

In [ ]:
variables = {}
for v in scenario.model.variables():
    variables[v.name] = v.value()


In [ ]:
var_df = pd.Series(variables).reset_index()
var_df.rename(columns={0:'value'},inplace=True)

var_df['parameter'] = var_df['index'].str.split('_').str[0]
var_df

In [ ]:
time_results = var_df.loc[var_df['parameter'].isin(['storagelevel','charge','discharge'])].copy()

time_results['timestep'] = time_results['index'].str.split('_').str[-1]
time_results['timestep'] = pd.to_numeric(time_results['timestep'])

time_results['time'] = df.index[0]+pd.to_timedelta(time_results['timestep'] + start_date_hours, unit='H')
time_results=time_results.sort_values(by=['parameter','time'])

In [ ]:
plot_df = time_results.pivot(index='time',columns='parameter',values='value')
plot_df['charge_cumulative'] = plot_df['charge'].cumsum()
plot_df['discharge_cumulative'] = plot_df['discharge'].cumsum()


plot_df = pd.concat([plot_df,df['Renewable Production']],axis=1)
plot_df['Load'] = load
plot_df['Curtailment_final'] = plot_df['Renewable Production'] + plot_df['discharge'] - plot_df['charge'] - load 
plot_df.describe()

In [ ]:
px.line(plot_df,title=f'Storage dispatch. Power {round(variables["power"],0)} capacity {round(variables["capacity"],0)}')

In [ ]:
plot_df['net charge'] = -plot_df['charge']

generation_params = [
    'net charge',
    'discharge',
    'Renewable Production'
]
px.area(plot_df[generation_params],title='Generation minus storage charging')

In [ ]:
px.area(plot_df[generation_params].resample('1m').mean(),title='Monthly generation and storage averages')